# COGS 108 - Data Checkpoint

# Names

- Annie Huang
- Andrew Cheng
- Lauren Sidarto
- Isaac Otero

<a id='research_question'></a>
# Research Question
What features [time, number of ingredients, number of steps, etc.] of a dinner recipe are most predictive of its star rating?

# Dataset(s)
Dataset: recipes.csv 

We utilized web scraping on http://allrecipes.com/ in order to build our dataset, so there is no link. The file name is recipes.csv. 

1386 observations after cleaning the dataset. 

This dataset contains recipe information from allrecipes.com. We included information on factors that we believed could have a significant influence on recipe star rating, such as number of ingredients, number of steps in a recipe, and the word count per step. 

We do not plan to use multiple datasets.

# Setup

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import time
# from torrequest import TorRequest
import numpy as np
import pandas as pd

# Scraping the Data


In [30]:
#constants
url = 'http://allrecipes.com/recipes/'
urls_to_scrape = []
categories = []

In [31]:
# collecting urls to scrape
ends = ['17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/',
        '15436/everyday-cooking/one-pot-meals/',
        '455/everyday-cooking/more-meal-ideas/30-minute-meals/',
        '94/soups-stews-and-chili/',
        '16099/everyday-cooking/comfort-food/',
        '80/main-dish/',
        '22992/everyday-cooking/sheet-pan-dinners/',
        '78/breakfast-and-brunch/',
        '17561/lunch/',
        '84/healthy-recipes/',
        '76/appetizers-and-snacks/',
        '96/salad/',
        '81/side-dish/',
        '156/bread/',
        '77/drinks/',
        '79/desserts/',
        '201/meat-and-poultry/chicken/',
        '200/meat-and-poultry/beef/',
        '205/meat-and-poultry/pork/',
        '93/seafood/',
        '1058/fruits-and-vegetables/fruits/',
        '95/pasta-and-noodles/',
        '1059/fruits-and-vegetables/vegetables/',
        '17562/dinner/',
       ]

cat = ['5-ingredient-main-dishes',
        'one-pot-meals',
        '30-minute-meals',
        'soups-stews-and-chili',
        'comfort-food',
        'main-dish',
        'sheet-pan-dinners',
        'breakfast-and-brunch',
        'lunch',
        'healthy-recipes',
        'appetizers-and-snacks',
        'salad',
        'side-dish',
        'bread',
        'drinks',
        'desserts',
        'chicken',
        'beef',
        'pork',
        'seafood',
        'fruits',
        'pasta-and-noodles',
        'vegetables',
        'dinner',
      ]


for e in ends:
    f = requests.get(url + e)
    soup = BeautifulSoup(f.text)

    try: 
        recipe_cards = soup.find('div', id="mntl-taxonomysc-article-list-group_1-0").findAll('a',href=True)
    except:
        recipe_cards = soup.find('div', id="tax-sc__recirc-list_1-0").findAll('a',href=True)

        
    temp = []
    for i in recipe_cards:
        temp.append(i['href'])
        urls_to_scrape.append(i['href'])
        categories.append(cat[ends.index(e)])

In [8]:
# save gathered urls to a csv. quality of life stuff
with open('urls_to_scrape.csv', 'w') as f:
    for x in urls_to_scrape:
        f.write(x+', ')

In [37]:
#read urls from csv
import csv
with open('urls_to_scrape.csv', newline='') as f:
    reader = csv.reader(f)
    urls_to_scrape = list(reader)
urls_to_scrape = urls_to_scrape[0][:-1] #the last line is a space so remove last url

IndexError: list index out of range

In [40]:
def scrape_recipe(u): 
#scraping individual parts of a recipe
    #recipe title
    f = requests.get(u)
    soup = BeautifulSoup(f.text)
    info = {}
    
    try:
        info["recipe_title"] = soup.findAll('h1')[0].text.strip()

        #review, rating
        temp = soup.findAll('div', {'id':'mntl-recipe-review-bar_1-0'})[0].text.split()
        info["star_rating"] = float(temp[0])
        info["rating_count"] = temp[1]
        info["review_count"] = temp[2]

        #cook time info
        temp = soup.findAll('div', {'class':'mntl-recipe-details__item'})
        for a in temp:
            spl = a.text.strip().split(":")
            info[spl[0]] = spl[1].strip()

        #recipe writeup
        writeup = ''
        for b in soup.find('div', {'id':'mntl-recipe-intro__content_1-0'}).findAll('p'):
            writeup += b.text.strip() + ' '
        info["writeup"] = writeup

        #ingredients
        ingr = []
        for c in soup.findAll('li',{'class':'mntl-structured-ingredients__list-item'}):
            ingr.append(c.text.strip())
        info["ingr"] = ingr

        #steps
        steps = []
        for step in soup.find('div',{'id':'recipe__steps_1-0'}).findAll('p',{'class':'comp mntl-sc-block mntl-sc-block-html'}):
            steps.append(step.text.strip())
        info['steps'] = steps

        #nutrition info - we decided against this
#         x=soup.findAll('td', {'class':'mntl-nutrition-facts-summary__table-cell'})
#         for d in range(len(x)-1):
#             if (d%2 ==0):
#                 info[x[d+1].text] = x[d].text
#             d+=1

    except:
        #if the recipe can't be scraped (e.g. it's one of those top 10 articles, 
        #or an article with no rating, then print the link)
        print(u) 
    return info



In [42]:
#scrape all recipes
rows = []
c = 0
for u in urls_to_scrape: 
    if (c%50 == 0):
        print(c)
        
    temp = scrape_recipe(u)
    temp['category'] = categories[urls_to_scrape.index(u)]
    rows.append(temp)
    c+=1

df = pd.DataFrame.from_dict(rows)
df.head()
df.to_csv('recipes.csv')

0
50
100
https://www.allrecipes.com/gallery/easy-one-dish-pork-chop-dinners/
https://www.allrecipes.com/recipe/212922/filipino-rice-arroz-valenciana/
150
200
250
300
https://www.allrecipes.com/gallery/the-best-tilapia-recipes/
https://www.allrecipes.com/gallery/best-traditional-christmas-dishes/
https://www.allrecipes.com/gallery/pulled-pork-sides/
https://www.allrecipes.com/gallery/make-and-freeze-casseroles/
https://www.allrecipes.com/gallery/side-dishes-hot-dogs/
https://www.allrecipes.com/gallery/stir-fry-sauces/
350
https://www.allrecipes.com/gallery/side-dishes-pork-chops/
https://www.allrecipes.com/gallery/the-best-sheet-pan-chicken-dinners/
https://www.allrecipes.com/gallery/healthy-sheet-pan-recipes-for-winter/
https://www.allrecipes.com/gallery/chicken-thigh-sheet-pan-dinner-recipes/
https://www.allrecipes.com/gallery/sheet-pan-shrimp-dinners/
https://www.allrecipes.com/gallery/super-easy-sheet-pan-suppers/
https://www.allrecipes.com/gallery/sheet-pan-fish-recipes/
400
https:

In [77]:
#read urls
df = pd.read_csv('recipes.csv')
df.head()

,Unnamed: 0,recipe_title,star_rating,rating_count,review_count,Prep Time,Cook Time,Total Time,Servings,Yield,writeup,ingr,steps,category,Additional Time,Chill Time,Marinate Time,Rest Time
0,0,World's Best Honey Garlic Pork Chops,4.4,(645),475,10 mins,15 mins,25 mins,6.0,6 servings,When you're in need of a quick and easy dinner...,"['½ cup ketchup', '2 ⅔ tablespoons honey', '2 ...",['Preheat grill for medium heat and lightly oi...,5-ingredient-main-dishes,NaN,NaN,NaN,NaN
1,1,Pan-Seared Tilapia,4.7,(789),560,5 mins,10 mins,15 mins,4.0,4 fillets,If you're looking for an easy weeknight dinner...,"['4 (4 ounce) tilapia fillets', 'salt and grou...",['Rinse tilapia fillets in cold water and pat ...,5-ingredient-main-dishes,NaN,NaN,NaN,NaN
2,2,Easy Baked Chicken Thighs,4.2,(194),135,5 mins,30 mins,35 mins,4.0,NaN,NaN,"['4 chicken thighs', '4 teaspoons garlic powd...",['Preheat the oven to 375 degrees F (190 degre...,5-ingredient-main-dishes,NaN,NaN,NaN,NaN
3,3,Chicken Cordon Bleu,4.6,"(2,668)","1,849",10 mins,35 mins,45 mins,4.0,4 servings,Don't feel like going out tonight? You can eas...,"['nonstick cooking spray', '4 skinless, bonel...",['Preheat oven to 350 degrees F (175 degrees C...,5-ingredient-main-dishes,NaN,NaN,NaN,NaN
4,4,Fluffy Microwave Scrambled Eggs,4.7,(60),58,5 mins,5 mins,10 mins,2.0,2 servings,Your microwave is the key to irresistibly fluf...,"['4 eggs', '¼ cup milk', '⅛ teaspoon salt']",['Break the eggs into a microwave-proof mixing...,5-ingredient-main-dishes,NaN,NaN,NaN,NaN


# Data Cleaning
We started with data scraped from allrecipes.com. We removed all recipes that had a Null value for star rating, as we are primarily interested in the relationship between star rating and various other recipe characteristics. For any Null values in other columns, we imputed them with zeros or replaced them with the average for that category/column, depending on what made sense contextually. For example, Null review counts were replaced with zeros, while Null recipe cook times were replaced with the average cook time for the recipe category. We converted all numerical columns into int or float types and made sure all numbers utilized uniform units. For example, we converted all hours and days under cook time into minutes. We replaced the ingredient list and list of steps with the ingredient/step count, and added a column for word count per step.

In [78]:
df.shape

(1440, 18)

In [79]:
#drop rows with null ratings
df = df[df['star_rating'].notna()]

#strip rating and review columns 
df['rating_count'] = df['rating_count'].str.slice(1,-1)
df['rating_count'] = df['rating_count'].str.replace(',','', regex = True)
df['review_count'] = df['review_count'].str.replace(',','', regex = True)

#turn rating and review columsn into floats
replace = []
df = df.reset_index(drop = True)
for i in df.index:
    temp = df['ingr'][i]
    temp = temp.split('\'')[1::2]
    replace.append(len(temp))
for i in range(len(replace)):
    df.iloc[i, 11] = replace[i]
    
df = df.astype({'rating_count': 'float64', 'review_count': 'float64', 'ingr': 'int'})

In [80]:
#convert prep, cook, and total time columns into minutes 

# Param: Time is string or null (mins, hrs, days)
# Return: Total time in minutes
def time_to_int(time):
    #Check if not string will have to null input later
    if type(time) != str:
        return np.NaN
    
    #Gets the digits of all times in mins, hrs, days, etc.
    times = re.findall('\d+', time)[::-1]
    
    #Sums up each time and multiplies it by the number of mins in that time
    return sum([int(times[i]) * (60 ** i) for i in range(len(times))])

#Converts prep time, cook time, and total time to int values
df['Prep Time'] = df['Prep Time'].apply(time_to_int)
df['Cook Time'] = df['Cook Time'].apply(time_to_int)
df['Total Time'] = df['Total Time'].apply(time_to_int)

In [81]:
#ftotal_steps_character and number_of_steps
df['steps_character']=df['steps'].str.strip('[]')
df['steps_character']=df['steps_character'].str.replace("'"," ")
df['steps_character']=df['steps_character'].str.replace(','," ")
df['steps_character']=df['steps_character'].str.replace(';'," ")
df['steps_character']=df['steps_character'].str.replace(" ", "")
df['total_steps_character']=df['steps_character'].str.len()
df=df.drop(columns=['steps_character'])


df['steps']=df['steps'].str.split("',")
df['number_of_steps']=df['steps'].str.len()

In [82]:
#finding average_step_length 
def find_len_steps(steps):
    s=0
    try:
        for x in steps:
            s=s+len(x)
        s=s/len(steps)
    except:
        print(steps)
    return round(s,2)

df['average_step_len']=df['steps'].apply(lambda steps: find_len_steps(steps))
df=df.drop(columns=['steps'])


In [83]:
#turning writeup into length
df['writeup']=df['writeup'].fillna('')
df.writeup = df.writeup.apply(lambda x: len(x))

In [84]:
#drop unused columns: index, additional time, chill time, marinate time, rest time, yield
df = df.drop(columns=['Unnamed: 0','Additional Time','Chill Time','Marinate Time','Rest Time','Yield'], axis=1)

#rename columns to have consistent names
df = df.rename({"Prep Time":"prep_time","Cook Time":"cook_time","Total Time":"total_time", "Servings":"servings"},axis=1)

In [85]:
df.head()

,recipe_title,star_rating,rating_count,review_count,prep_time,cook_time,total_time,servings,writeup,ingr,category,total_steps_character,number_of_steps,average_step_len
0,World's Best Honey Garlic Pork Chops,4.4,645.0,475.0,10.0,15.0,25.0,6.0,3602,5,5-ingredient-main-dishes,342,3,143.33
1,Pan-Seared Tilapia,4.7,789.0,560.0,5.0,10.0,15.0,4.0,2580,8,5-ingredient-main-dishes,475,5,117.60
2,Easy Baked Chicken Thighs,4.2,194.0,135.0,5.0,30.0,35.0,4.0,0,3,5-ingredient-main-dishes,299,3,124.33
3,Chicken Cordon Bleu,4.6,2668.0,1849.0,10.0,35.0,45.0,4.0,4173,7,5-ingredient-main-dishes,547,5,135.80
4,Fluffy Microwave Scrambled Eggs,4.7,60.0,58.0,5.0,5.0,10.0,2.0,2589,3,5-ingredient-main-dishes,318,2,199.00


Null imputation

In [87]:
#all time columns have null values
df.isna().any()

recipe_title             False
star_rating              False
rating_count             False
review_count             False
prep_time                 True
cook_time                 True
total_time                True
servings                 False
writeup                  False
ingr                     False
category                 False
total_steps_character    False
number_of_steps          False
average_step_len         False
dtype: bool

In [88]:
#some of the null rows under "prep_time" column
df[df['prep_time'].isna()].head()

,recipe_title,star_rating,rating_count,review_count,prep_time,cook_time,total_time,servings,writeup,ingr,category,total_steps_character,number_of_steps,average_step_len
67,Scrambled Eggs in a Mug,4.5,43.0,32.0,NaN,5.0,5.0,1.0,0,3,one-pot-meals,160,2,99.00
316,Brined and Roasted Whole Turkey,4.7,23.0,17.0,NaN,NaN,800.0,10.0,0,8,main-dish,1292,7,226.43
458,Egg in a Hole,4.7,358.0,301.0,NaN,5.0,5.0,1.0,0,4,breakfast-and-brunch,281,2,177.00
710,Buffalo Chicken Wing Sauce,4.8,845.0,649.0,NaN,5.0,5.0,8.0,3244,7,side-dish,229,1,294.00
898,Soft Pumpkin Cookies,3.3,62.0,48.0,NaN,NaN,NaN,12.0,0,9,desserts,187,4,59.50


We decided it would make sense to impute NaNs with the mean of the cook time under each category of food. For example, a NaN time for a drink will have the average cook time for a drink.

In [89]:
#Gets the mean time for each category for prep cook and total time
mean_time = df[['category','prep_time', 'cook_time', 'total_time']].groupby('category').transform('mean').round(2)
#Imputes the mean times of each category for the missing values
df = df.fillna(mean_time)
df.isna().any()

recipe_title             False
star_rating              False
rating_count             False
review_count             False
prep_time                False
cook_time                False
total_time               False
servings                 False
writeup                  False
ingr                     False
category                 False
total_steps_character    False
number_of_steps          False
average_step_len         False
dtype: bool

In [90]:
df.shape

(1386, 14)

## EDA

In [84]:
df.describe()

,star_rating,rating_count,review_count,Prep Time,Cook Time,Total Time,Servings,ingr
count,1386.000000,1386.000000,1386.000000,1376.000000,1235.000000,1384.000000,1386.000000,1386.000000
mean,4.590043,1063.104618,791.934343,15.023256,34.012146,100.820809,8.562771,9.013709
std,0.222267,2005.081219,1466.906558,9.156310,52.999395,405.371516,10.983185,4.063995
min,2.300000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.500000,109.250000,85.000000,10.000000,10.000000,20.000000,4.000000,6.000000
50%,4.600000,433.000000,337.000000,15.000000,20.000000,40.000000,6.000000,8.000000
75%,4.700000,1174.750000,882.000000,20.000000,35.000000,80.000000,9.000000,11.000000
max,5.000000,19800.000000,14420.000000,90.000000,625.000000,11535.000000,272.000000,28.000000
